In [1]:
import numpy as np
import scipy.ndimage as ndimage


In [8]:
class DiffMatrix():
    
    def __init__(self,
                size= 300,
                diffusion_characteristics = 'default'):
#       Reaction caracteristics
        if diffusion_characteristics == 'default':
            self.diff_a = 1
            self.diff_b = 0.5
            self.feed_rate = 0.055
            self.kill_rate = 0.062
            self.timestep = 1
            self.laplace_window = 3
        
        self.size = size
        self.matrix_shape = (int(self.size), int(self.size))
        self.matrix_a = self.initiate_matrix()
        self.matrix_b = self.initiate_matrix()
        self.matrix_a_new = self.initiate_matrix()
        self.matrix_b_new = self.initiate_matrix()
    
    def initiate_matrix(self):
        matrix = np.zeros(self.matrix_shape)
        return matrix
    
    def fill_matrix_tiny(self):
        for matrix in (self.matrix_a, self.matrix_b):
            matrix[1][2] = 0.3
            matrix[1][3] = 0.3
            matrix[2][2] = 0.3

    def fill_matrix_random(self):
        self.matrix_a = np.random.rand(*self.matrix_shape)
        self.matrix_b = np.random.rand(*self.matrix_shape)
        
    def fill_matrix_propper(self):
        self.matrix_a = np.ones(self.matrix_shape)
        self.matrix_b = np.zeros(self.matrix_shape)
        
        start_b = int(0.45 * self.size)
        end_b = int(0.55 * self.size)
        self.matrix_b[start_b:end_b,start_b:end_b] = 1

    
    def switch_new(self):
        self.matrix_a = self.matrix_a_new.copy()
        self.matrix_b = self.matrix_b_new.copy()
    
    def feed(self):
        # create a up until 1
        self.matrix_a_new += self.feed_rate * (1- self.matrix_a)
        #bestroy b if any
        self.matrix_b_new -= (self.feed_rate + self.kill_rate) * self.matrix_b
    
    def react(self):
        #consume a and turn it into b
        reaction_rate = self.matrix_a * self.matrix_b * self.matrix_b
        self.matrix_a_new -= reaction_rate
        self.matrix_b_new += reaction_rate    
        
    def diffuse(self):
        dm.matrix_a_new = ndimage.generic_filter(dm.matrix_a, self.lap_filter, size=(3,3))
        dm.matrix_b_new = ndimage.generic_filter(dm.matrix_b, self.lap_filter, size=(3,3))
        
        
    def lap_filter(self,x):
        x = x.reshape((self.laplace_window,self.laplace_window))
        filter_total = x[1][1] * -1
        # cross:
        filter_total += x[1][0] * 0.2
        filter_total += x[0][1] * 0.2
        filter_total += x[1][2] * 0.2
        filter_total += x[2][1] * 0.2

        #diag
        filter_total += x[0][0] * 0.05
        filter_total += x[2][0] * 0.05
        filter_total += x[0][2] * 0.05
        filter_total += x[2][2] * 0.05
        return filter_total

    def _next(self):

        self.diffuse()
        self.feed()
        self.react()

        self.switch_new()
#         self.matrix_a_new = self.initiate_matrix()
#         self.matrix_b_new = self.initiate_matrix()
    


In [3]:
A = np.reshape(range(16),(4,4))

In [5]:
A = np.reshape([0.0,0,0,0,1,0,0,0,0],(3,3))
B = np.reshape([0.0,0,0,0,0,0,0,0,0],(3,3))
B = ndimage.generic_filter(A, lap_filter, size=(3,3))

In [9]:
dm = DiffMatrix(100)
dm.fill_matrix_propper()

In [6]:
dm.matrix_a

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]])

In [10]:
for i in range(10):
    dm._next()

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:57: RuntimeWarning: overflow encountered in multiply


In [62]:
dm.matrix_a.mean()

nan